In [1]:
import pandas as pd
import copy
import os
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
import copy
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
from skyfield import api
import pandas as pd
from skyfield import almanac
from datetime import datetime
ts = api.load.timescale()
eph = api.load('de421.bsp')
bluffton = api.wgs84.latlon(+43.6532, -79.3832)
times={'Date':[],'Sunrise':[],'Sunset':[]}

In [2]:
# Getting the daylight for observational period
t0 = ts.utc(2002,1, 1)
t1 = ts.utc(2018, 1, 1)
t, y = almanac.find_discrete(t0, t1, almanac.sunrise_sunset(eph, bluffton))
# Breaking down each specific portion
for i in range(0,len(t),2):
    times['Date'].append(t.utc_iso()[i][0:10])
    times['Sunrise'].append(t.utc_iso()[i][11:19])
    times['Sunset'].append(t.utc_iso()[i+1][11:19])
# Changing the format for sunrise and sunset
for i in range(0,len(times['Date'])):
    datedata=datetime.strptime(times['Sunrise'][i],'%H:%M:%S')
    times['Sunrise'][i]=datedata
    datedata=datetime.strptime(times['Sunset'][i],'%H:%M:%S')
    times['Sunset'][i]=datedata
times=pd.DataFrame.from_dict(times)
times['Daylight']=0
for i in range(0,len(times['Date'])):
    times['Daylight'][i]=times['Sunset'][i]-times['Sunrise'][i]
times['Daylight']= times['Daylight'].astype(str).map(lambda x: x[7:])
results=times['Daylight'].astype(str)
results=results.str.replace('+','')
times['Daylight']=results
times['Date']=pd.to_datetime(times['Date'])

In [3]:
SM = pd.read_excel(r'Path\To\File\SoilMoisturePolygon.xlsx')
VD = pd.read_excel(r'Path\To\File\VegetationDataPolygon.xlsx')
ClimateData = pd.read_excel(r'Path\To\File\ClimateDataNew2023.xlsx')
StreamflowData = pd.read_excel(r'Path\To\File\Streamflow2023.xlsx')
CompiledMosqCount = pd.read_excel(r'Path\To\Mosquito_Dataset.xlsx') # Mosquito Dataset must be requested from source 


In [6]:
import geopandas as gpd
# Getting datapoints from mosquito long and lat that are only within the GTA region
shapefile_path = r"Path\To\File\GTA.shp"
gdf = gpd.read_file(shapefile_path)

excel_path = r'Path\To\Mosquito_Dataset.xlsx'
df = pd.read_excel(excel_path)
geometry = gpd.points_from_xy( df['Longitude'],df['Latitude'])
gdf_points = gpd.GeoDataFrame(df, geometry=geometry)
points_inside = gpd.sjoin(gdf_points, gdf, how='inner', op='intersects')
latitude_inside = points_inside['Latitude']
longitude_inside = points_inside['Longitude']
CompiledMosqCount=points_inside

In [7]:
StreamflowData['Date']=pd.to_datetime(StreamflowData['Date'])
StreamflowData['Year']=StreamflowData['Date'].dt.to_period('Y')
StreamflowData['Month']=StreamflowData['Date'].dt.to_period('M')
StreamflowDatamod=copy.deepcopy(StreamflowData)
StreamflowDatamod=StreamflowDatamod.drop(columns=['Unnamed: 0'])

In [9]:
# Cleaning SM data. 
SM['Date']=pd.to_datetime(SM['Date'])
SM['Year']=SM['Date'].dt.to_period('Y')
SM['Month']=SM['Date'].dt.to_period('M')
idx = pd.date_range('2002-01-01', '2017-12-31')
SMtest=copy.deepcopy(SM)
SMtest.set_index('Date', inplace=True)
SMtest = SMtest.reindex(idx, fill_value=np.nan)
SMtest = SMtest.reset_index()
SMtest.rename(columns = {'index':'Date'}, inplace = True)
SMtest['Date']=pd.to_datetime(SMtest['Date'])
SMtest['Year']=SMtest['Date'].dt.year
SMtest['Month']=SMtest['Date'].dt.month
SMtest=SMtest.drop(columns=['Unnamed: 0','Count'])
SMtest['SM']=SMtest['SM'].interpolate(method='linear')
SMtest['Moving SM']=SMtest['SM']
SMtest.fillna(method='bfill', inplace=True)
# SMtest.fillna(method='ffill', inplace=True)

In [11]:
Mosqcount=copy.deepcopy(CompiledMosqCount)

# Drop the unnecessary data and add in indep mon, day, yr
CompiledMosqCount=CompiledMosqCount.drop(columns=['index_right', 'GEODB_OID', 'OBJECTID',
                                                  'OBJECT_ID', 'GUT_NUMBER', 'SHAPE_AREA', 'SHAPE_LEN','Week Code','Health Unit','Latitude','Longitude',
                                                  'Trap Type','Hotspot','Species','Physiological Status','Date Extracted',
                                                  'Result','Comments','Site Code', 'Unnamed: 5'])

CompiledMosqCount=CompiledMosqCount.dropna()

# We decided to take the daily mean and not the sum. This is the average number collected per day across the gta region
CompiledMosqCount=CompiledMosqCount.groupby(['Date Collected'],as_index=False).mean()
CompiledMosqCount['Month']=pd.DatetimeIndex(CompiledMosqCount['Date Collected']).month
CompiledMosqCount['Day']=pd.DatetimeIndex(CompiledMosqCount['Date Collected']).day
CompiledMosqCount['Year']=pd.DatetimeIndex(CompiledMosqCount['Date Collected']).year
CompiledMosqCount['Date']=CompiledMosqCount['Date Collected']
CompiledMosqCount=CompiledMosqCount.drop(columns=['Date Collected'])

In [12]:
CompiledMosqCount

,Year,# of Mosquitoes,Month,Day,Date
0,2002,2.333333,5,31,2002-05-31
1,2002,4.785714,6,4,2002-06-04
2,2002,3.347826,6,6,2002-06-06
3,2002,7.741935,6,11,2002-06-11
4,2002,3.903226,6,13,2002-06-13
...,...,...,...,...,...
712,2017,3.058824,9,18,2017-09-18
713,2017,5.112150,9,19,2017-09-19
714,2017,7.038835,9,20,2017-09-20
715,2017,5.991150,9,26,2017-09-26


In [13]:
CompiledMosqCount2=copy.deepcopy(CompiledMosqCount)

# Fill in the missing dates as 0
idx = pd.date_range('2002-05-31', '2017-10-11')
CompiledMosqCount2.set_index('Date', inplace=True)
CompiledMosqCount2 = CompiledMosqCount2.reindex(idx, fill_value='0')
CompiledMosqCount2 = CompiledMosqCount2.reset_index()
CompiledMosqCount2['Date']=CompiledMosqCount2['index']
CompiledMosqCount2=CompiledMosqCount2.drop(columns=['index'])

for i, row in CompiledMosqCount2.iterrows():
    CompiledMosqCount2['Year'][i]=CompiledMosqCount2['Date'][i].year
    CompiledMosqCount2['Day'][i]=CompiledMosqCount2['Date'][i].day
    CompiledMosqCount2['Month'][i]=CompiledMosqCount2['Date'][i].month
CompiledMosqCount2['# of Mosquitoes']=CompiledMosqCount2['# of Mosquitoes'].astype(float)

# This period os the mosquito season, so we don't need 0's in these months
CompiledMosqCount2.drop(CompiledMosqCount2[(CompiledMosqCount2['Month'] ==5) & (CompiledMosqCount2['# of Mosquitoes'] ==0)].index, inplace=True)
CompiledMosqCount2.drop(CompiledMosqCount2[(CompiledMosqCount2['Month'] ==6) & (CompiledMosqCount2['# of Mosquitoes'] ==0)].index, inplace=True)
CompiledMosqCount2.drop(CompiledMosqCount2[(CompiledMosqCount2['Month'] ==7) & (CompiledMosqCount2['# of Mosquitoes'] ==0)].index, inplace=True)
CompiledMosqCount2.drop(CompiledMosqCount2[(CompiledMosqCount2['Month'] ==8) & (CompiledMosqCount2['# of Mosquitoes'] ==0)].index, inplace=True)
CompiledMosqCount2.drop(CompiledMosqCount2[(CompiledMosqCount2['Month'] ==9) & (CompiledMosqCount2['# of Mosquitoes'] ==0)].index, inplace=True)
CompiledMosqCount2.drop(CompiledMosqCount2[(CompiledMosqCount2['Month'] ==10) & (CompiledMosqCount2['# of Mosquitoes'] ==0)].index, inplace=True)
CompiledMosqCount2=CompiledMosqCount2.reset_index()

# Data during these months was nonexistent, however, just to be sure.
for i in range(0,len(CompiledMosqCount2)):
    if CompiledMosqCount2['Month'][i]==4:
        CompiledMosqCount2['# of Mosquitoes'][i]=0

for i in range(0,len(CompiledMosqCount2)):
    if CompiledMosqCount2['Month'][i]==3:
        CompiledMosqCount2['# of Mosquitoes'][i]=0

for i in range(0,len(CompiledMosqCount2)):
    if CompiledMosqCount2['Month'][i]==2:
        CompiledMosqCount2['# of Mosquitoes'][i]=0

for i in range(0,len(CompiledMosqCount2)):
    if CompiledMosqCount2['Month'][i]==1:
        CompiledMosqCount2['# of Mosquitoes'][i]=0

In [16]:
window_size = 7

# 7-day rolling average for datasets. All these datasets are daily data, so we don't need further manipulation.
numbers_series_climatedata_temp = pd.Series(ClimateData['Mean Temp (°C)'])
windows_climatedata_temp = numbers_series_climatedata_temp.rolling(window_size)
moving_climatedata_temp = windows_climatedata_temp.mean()
ClimateData['Moving Mean Temp']=moving_climatedata_temp
ClimateData['Date']=pd.to_datetime(ClimateData[['Year','Month','Day']])
ClimateData['Moving Mean Temp'][0:6]=ClimateData['Mean Temp (°C)'][0:6]

numbers_series_climatedata_precip = pd.Series(ClimateData['Total Precip (mm)'])
windows_climatedata_precip = numbers_series_climatedata_precip.rolling(window_size)
moving_climatedata_precip = windows_climatedata_precip.mean()
ClimateData['Moving Total Precip']=moving_climatedata_precip
ClimateData['Moving Total Precip'][0:6]=ClimateData['Total Precip (mm)'][0:6]

numbers_series_climatedata_rain = pd.Series(ClimateData['Total Rain (mm)'])
windows_climatedata_rain= numbers_series_climatedata_rain.rolling(window_size)
moving_climatedata_rain = windows_climatedata_rain.mean()

ClimateData['Moving Total Rain']=moving_climatedata_rain
ClimateData['Moving Total Rain'][0:6]=ClimateData['Total Rain (mm)'][0:6]

# LAI was monthly average
VD['Moving LAI']=VD['LAI']
VD=VD.drop(columns=['Unnamed: 0'])

StreamflowDataReal=copy.deepcopy(StreamflowDatamod)
numbers_series_StreamflowDatamod = pd.Series(StreamflowDatamod['Outflow (m^3/s)'])
windows_StreamflowData= numbers_series_StreamflowDatamod.rolling(window_size)
moving_StreamflowData = windows_StreamflowData.mean()
StreamflowDataReal['Moving Outflow (m^3/s)']=moving_StreamflowData
StreamflowDataReal['Moving Outflow (m^3/s)'][0:6]=StreamflowDatamod['Outflow (m^3/s)'][0:6]

In [17]:
CompiledMosqCount2['Day Gaps']=CompiledMosqCount2['Date']
# Checking how big the gaps are between datapoints
for i in range(1,len(CompiledMosqCount2)):
    CompiledMosqCount2['Day Gaps'][i]=(CompiledMosqCount2['Date'][i]-CompiledMosqCount2['Date'][i-1]).days
CompiledMosqCount2['Day Gaps'][0]=0

In [18]:
CompiledMosqCount2[CompiledMosqCount2['Day Gaps']>4]

,index,Year,# of Mosquitoes,Month,Day,Date,Day Gaps
3,11,2002,7.741935,6,11,2002-06-11,5
5,18,2002,11.488095,6,18,2002-06-18,5
7,25,2002,7.135484,6,25,2002-06-25,5
21,46,2002,13.518072,7,16,2002-07-16,5
24,53,2002,7.833333,7,23,2002-07-23,5
...,...,...,...,...,...,...,...
3421,5562,2017,9.010471,8,22,2017-08-22,6
3423,5569,2017,9.581152,8,29,2017-08-29,6
3425,5577,2017,7.501946,9,6,2017-09-06,7
3430,5589,2017,3.058824,9,18,2017-09-18,5


In [19]:
CompiledMosqCount2[CompiledMosqCount2['Day Gaps']>5]

,index,Year,# of Mosquitoes,Month,Day,Date,Day Gaps
45,96,2002,7.512821,9,4,2002-09-04,6
65,138,2002,1.210526,10,16,2002-10-16,6
66,144,2002,1.000000,10,22,2002-10-22,6
67,154,2002,0.000000,11,1,2002-11-01,10
248,349,2003,1.000000,5,15,2003-05-15,15
...,...,...,...,...,...,...,...
3417,5549,2017,13.848387,8,9,2017-08-09,7
3421,5562,2017,9.010471,8,22,2017-08-22,6
3423,5569,2017,9.581152,8,29,2017-08-29,6
3425,5577,2017,7.501946,9,6,2017-09-06,7


In [20]:
CompiledMosqCount2[CompiledMosqCount2['Day Gaps']>7]

,index,Year,# of Mosquitoes,Month,Day,Date,Day Gaps
67,154,2002,0.000000,11,1,2002-11-01,10
248,349,2003,1.000000,5,15,2003-05-15,15
315,519,2003,0.000000,11,1,2003-11-01,11
497,732,2004,13.571429,6,1,2004-06-01,32
557,885,2004,0.000000,11,1,2004-11-01,19
738,1110,2005,8.630252,6,14,2005-06-14,45
792,1250,2005,0.000000,11,1,2005-11-01,18
973,1473,2006,11.250000,6,12,2006-06-12,43
1026,1615,2006,0.000000,11,1,2006-11-01,28
1207,1834,2007,2.000000,6,8,2007-06-08,39


In [21]:
# 7 day rolling average
def custom_rolling_avg(df, target, date, window=7):
    df['# of Mosquitoes Rolling'] = None
    for i, row in df.iterrows():
        SumOfDays = 0
        TemporaryCount = 0
        RemainingDays = window
        # Looping backwards
        for j in range(i, -1, -1):
            if j != i:
                # Calculating how many days are between the two points. If the remaining days are greater than 7, it keeps going. Otherwise stop.
                DaysGap = (df.loc[j+1, date] - df.loc[j, date]).days
                RemainingDays -= DaysGap # subtract the day gap from the total days
            if RemainingDays >= 0:
                SumOfDays += df.loc[j, target] # adding the total number of mosquitoes across the window
                TemporaryCount += 1 # Adding the total number of days
            else:
                break
        df.at[i, '# of Mosquitoes Rolling'] = SumOfDays / TemporaryCount # calcualting the average number of mosquitoes across those days
custom_rolling_avg(CompiledMosqCount2, '# of Mosquitoes', 'Date')

In [22]:
CompiledMosqCount2

,index,Year,# of Mosquitoes,Month,Day,Date,Day Gaps,# of Mosquitoes Rolling
0,0,2002,2.333333,5,31,2002-05-31,0,2.333333
1,4,2002,4.785714,6,4,2002-06-04,4,3.559524
2,6,2002,3.347826,6,6,2002-06-06,2,3.488958
3,11,2002,7.741935,6,11,2002-06-11,5,5.291825
4,13,2002,3.903226,6,13,2002-06-13,2,4.997662
...,...,...,...,...,...,...,...,...
3430,5589,2017,3.058824,9,18,2017-09-18,5,4.805404
3431,5590,2017,5.112150,9,19,2017-09-19,1,4.637013
3432,5591,2017,7.038835,9,20,2017-09-20,1,5.413816
3433,5597,2017,5.991150,9,26,2017-09-26,6,6.047378


In [24]:
SMtest=SMtest.dropna()

In [25]:
# Merging everything
CumulativeData=pd.merge(StreamflowDataReal, SMtest, how = 'inner', left_on = ['Date'], right_on = ['Date'])
CumulativeData=pd.merge(VD, CumulativeData, how = 'inner', left_on = ['Date'], right_on = ['Date'])
CumulativeData=pd.merge(ClimateData, CumulativeData, how = 'inner', left_on = ['Date'], right_on = ['Date'])
CumulativeData=pd.merge(CompiledMosqCount2, CumulativeData, how = 'left', left_on = ['Date'], right_on = ['Date'])
CumulativeDatatest=CumulativeData.drop(columns=['index', 'Year_x', 'Month_x', 'Day_x',
       'Day Gaps', 'Year_y', 'Month_y', 'Day_y', 'Longitude (x)', 'Latitude (y)','Count', 'Year_x', 'Month_x', 'Year_y', 'Month_y','Unnamed: 0'])


In [27]:
timestuff=copy.deepcopy(times)


In [28]:
times['Daylight_s']=times['Daylight']
for i in range(0,len(times)):
    try:
        times['Daylight_s'][i]=int(times['Daylight'][i][0:2])*3600+int(times['Daylight'][i][3:5])*60+int(times['Daylight'][i][6:8])
    except ValueError:
        times['Daylight_s'][i]=int(times['Daylight'][i][0:3])*3600+int(times['Daylight'][i][4:6])*60+int(times['Daylight'][i][7:9])

# rolling daylight 
window_size = 7
numbers_series_Daylight = pd.Series(times['Daylight_s'])
windows_Daylight= numbers_series_Daylight.rolling(window_size)
moving_Daylight = windows_Daylight.mean()
times['Moving Daylight']=moving_Daylight
times['Moving Daylight'][0:6]=times['Daylight_s'][0:6]

In [29]:
CumulativeDatatest=pd.merge(times, CumulativeDatatest, how = 'inner', left_on = ['Date'], right_on = ['Date'])
CumulativeDatatest=CumulativeDatatest.drop(columns=['Sunrise','Sunset','Daylight'])


In [30]:
CumulativeDatatest['Timeskip']=CumulativeDatatest['Date']
for i in range(0,len(CumulativeDatatest)):
    try:
        CumulativeDatatest['Timeskip'][i]=CumulativeDatatest['Date'][i+1]-CumulativeDatatest['Date'][i]
        CumulativeDatatest['Timeskip'][i]=CumulativeDatatest['Timeskip'][i].total_seconds()
    except KeyError:
        CumulativeDatatest['Timeskip'][i]=0

In [31]:
CumulativeDatatest

,Date,Daylight_s,Moving Daylight,# of Mosquitoes,# of Mosquitoes Rolling,Mean Temp (°C),Total Precip (mm),Total Rain (mm),Moving Mean Temp,Moving Total Precip,Moving Total Rain,LAI,Moving LAI,Outflow (m^3/s),Moving Outflow (m^3/s),SM,Moving SM,Timeskip
0,2002-05-31,54712,54442.428571,2.333333,2.333333,20.411765,6.573913,6.573913,15.907096,4.275155,4.275155,0.480285,0.480285,81.232,47.555143,0.246492,0.246492,345600.0
1,2002-06-04,55014,54785.714286,4.785714,3.559524,10.605556,5.345833,5.369565,16.357236,4.856858,4.860248,0.912478,0.912478,45.700,54.019571,0.246492,0.246492,172800.0
2,2002-06-06,55143,54936.571429,3.347826,3.488958,14.463158,0.000000,0.000000,15.181898,1.944099,1.931056,0.912478,0.912478,40.393,54.770000,0.246492,0.246492,432000.0
3,2002-06-11,55397,55248.714286,7.741935,5.291825,21.994737,18.250000,18.947826,17.755096,2.743452,2.827950,0.912478,0.912478,28.121,34.758714,0.277600,0.277600,172800.0
4,2002-06-13,55471,55346.571429,3.903226,4.997662,18.573684,0.216667,0.217391,18.222807,2.901190,3.009938,0.912478,0.912478,59.791,40.771714,0.270698,0.270698,432000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3430,2017-09-18,44475,45006.714286,3.058824,4.805404,21.127273,0.183333,0.244444,19.374935,0.026190,0.034921,1.069761,1.069761,30.662,32.532571,0.253171,0.253171,86400.0
3431,2017-09-19,44296,44829.285714,5.112150,4.637013,22.210000,0.800000,0.511111,20.246494,0.140476,0.107937,1.069761,1.069761,31.208,31.933000,0.267820,0.267820,86400.0
3432,2017-09-20,44118,44651.714286,7.038835,5.413816,21.410000,0.018182,0.000000,20.849221,0.143074,0.107937,1.069761,1.069761,34.589,32.019286,0.268394,0.268394,518400.0
3433,2017-09-26,43049,43583.714286,5.991150,6.047378,24.730000,0.000000,0.000000,23.150794,0.002597,0.000000,1.069761,1.069761,28.346,30.751000,0.262452,0.262452,86400.0


In [33]:
times

,Date,Sunrise,Sunset,Daylight,Daylight_s,Moving Daylight
0,2002-01-01,1900-01-01 12:51:09,1900-01-01 21:51:20,09:00:11,32411,32411
1,2002-01-02,1900-01-01 12:51:12,1900-01-01 21:52:14,09:01:02,32462,32462
2,2002-01-03,1900-01-01 12:51:12,1900-01-01 21:53:10,09:01:58,32518,32518
3,2002-01-04,1900-01-01 12:51:10,1900-01-01 21:54:07,09:02:57,32577,32577
4,2002-01-05,1900-01-01 12:51:05,1900-01-01 21:55:07,09:04:02,32642,32642
...,...,...,...,...,...,...
5839,2017-12-27,1900-01-01 12:50:18,1900-01-01 21:47:23,08:57:05,32225,32172.285714
5840,2017-12-28,1900-01-01 12:50:33,1900-01-01 21:48:08,08:57:35,32255,32188.285714
5841,2017-12-29,1900-01-01 12:50:45,1900-01-01 21:48:55,08:58:10,32290,32209.142857
5842,2017-12-30,1900-01-01 12:50:55,1900-01-01 21:49:43,08:58:48,32328,32234.285714


In [34]:
CumulativeDatatest.to_excel('Path\To\Cumulative_Dataset_Polygon_Mosq_Update2.xlsx')
# This dataset is not included as it contains the mosquito data a well
times.to_excel('Path\To\DaylightDataset.xlsx')